# Numero de Nusselt

In [ ]:
!pip install gplearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Memoria/Data/

/content/drive/MyDrive/Memoria/Data


In [ ]:
import pandas as pd
import matplotlib
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import export_graphviz
import graphviz
import pandas as pd


In [ ]:
# cargando datos

# importacion de DF
file_path = 'df_n_25.txt'
df_n_25 = pd.read_csv(file_path, delimiter=',')

file_path = 'df_n_53.txt'
df_n_53 = pd.read_csv(file_path, delimiter=',')

file_path = 'df_n_74.txt'
df_n_74 = pd.read_csv(file_path, delimiter=',')

file_path = 'df_n_102.txt'
df_n_102 = pd.read_csv(file_path, delimiter=',')

In [ ]:
# definiendo conjuntos de train y test

# definiendo conjunto de train
df_n_train = pd.concat([df_n_25, df_n_74], ignore_index=True)

# separando entre x e y
y_train = df_n_train.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_train = df_n_train.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])


# definiendo conjunto de test

y_test = df_n_53.drop(columns=['Current','K','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','Rem','prandtl','colIndex'])
X_test = df_n_53.drop(columns=['Current','Flujo','t_viento','Diametro','col_fluido','col_celda','n_fluido','n_celda','colIndex','nusselt'])

y_train = y_train.values.ravel()

### Entrenar modelo original

In [ ]:
# entrenando modelo

# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=30,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div'])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    32.83      4.89184e+31       31          11.7311            11.78     17.84m
   1    17.97      6.65547e+15       31          9.48876          9.30918     10.14m
   2    26.49      4.08048e+09       27          6.99431          6.97922     10.41m
   3    30.02      1.92981e+13       27          7.00878          6.84901     11.08m
   4    29.69       1.2269e+11       27          6.93223          7.53787      9.77m
   5    32.47       4.5187e+07       25          6.67603          6.71544     10.18m
   6    35.73       1.3954e+09       67          3.30332          3.29504     10.31m
   7    38.75      1.40532e+10       57          3.24255          3.28773     11.61m
   8    38.52      7.44973e+07       57          3.23477          3.35773  

KeyboardInterrupt: 

In [ ]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 11.00072279485055
R^2: 0.9773198439795644


### Entrenar agregando  potencias V1

In [ ]:
# creando funciones auxiliares


# 0.5 * S ^(-0.2)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, 0.5*x1**(-0.2), 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=1)

# Re^(0.64) *Pr
def pot2(x1,x2):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, x1**(0.64)*x2, 0)
    return result

pot2_fn = make_function(function=pot2,
                        name='pot2',
                        arity=2)


In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=25,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    24.91      1.49917e+18       18          2.66988          2.67641     21.73m
   1    19.34      1.05018e+08       18          2.67266          2.65142     17.54m
   2    24.46      1.52122e+08       38           1.5917          1.64675     19.08m
   3    23.76      9.01816e+10       45          1.55893          1.59787     18.05m
   4    23.16      9.65336e+09       29          1.55417          1.57869     21.14m
   5    27.10       1.0189e+10       33          1.55048          1.55992     18.92m
   6    29.47       1.7731e+06       25          1.52482          1.57193     19.04m
   7    23.65           159348       25          1.52548          1.56594     14.12m
   8    21.43      1.64753e+06       25          1.52125          1.60405  

In [ ]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.800143692048582
R^2: 0.9901035586582426


# 0.4015568 * X1 ^(0.64) * (X0/X2)^(-2)

In [ ]:
# comparar con modelo de Rafael

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.3575 * Re **(0.64) * (S/Pr)**(-0.2)

In [ ]:
X_test

,K,Rem,prandtl
0,0.6,2824.604561,0.709843
1,0.6,2824.604561,0.709843
2,0.6,2352.302465,0.709842
3,0.6,2000.632597,0.709842
4,0.6,1878.735604,0.709842
...,...,...,...
8605,1.2,9267.885175,0.707768
8606,1.2,8959.387980,0.707705
8607,1.2,8691.558442,0.707625
8608,1.2,8537.950633,0.707554


In [ ]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9898656037772775 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 4.915560697152972 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.6305633774376995 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.8637994390677814*100 


### Entrenar agregando  potencias V2

In [ ]:
### Entrenar agregando  potencias V2

# 0.5 * S ^(-0.2) * Re ^0.64 * Pr
def pot1(x1, x2, x3):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0) & (x3>0), 0.5*x1**(-0.2)* x2 **(0.64) * x3, 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=3)


In [ ]:
X_train['prandtl'].describe()

count    15900.000000
mean         0.708010
std          0.001358
min          0.695312
25%          0.707104
50%          0.707981
75%          0.709256
max          0.710697
Name: prandtl, dtype: float64

In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    51.11      2.23472e+16       37          4.69737          4.72079     33.76m
   1    28.93      2.31285e+07       11          3.68762          3.68981     16.80m
   2    36.28      2.41273e+06       16          2.76631          2.74759     22.18m
   3    24.33       1.2282e+07        6           1.5527          1.64455     16.29m
   4    13.73      4.08384e+07        4          1.54242           1.6014     10.69m
   5    16.32           355479        4          1.53925          1.62987      9.97m
   6    12.83           301063       10          1.52804          1.55206      7.30m
   7     7.90      1.28664e+08        8          1.51811          1.56201      5.29m
   8     4.75      3.37881e+06        8          1.52101          1.53591  

In [ ]:
# comparar con modelo de Rafael

def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.5 * S **(-0.2)*Re **(0.64) * 0.660

In [ ]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9901974685565402 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 4.754593883752133 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.5605096310233906 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.720218831637322*100 


Se consiguen mejores metricas sin incluir el prantdl, ¿pero tendra sentido físico?

### Entrenar agregando  potencias V3

In [ ]:
# * S ^(-0.2)
def pot1(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(x1 > 0, x1**(-0.2), 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=1)

# 0.5*Re^(0.64) *Pr
def pot2(x1,x2):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0), 0.5*x1**(0.64)*x2, 0)
    return result

pot2_fn = make_function(function=pot2,
                        name='pot2',
                        arity=2)


In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    24.91      1.35864e+18        6          4.81152           4.8322     17.77m
   1    19.51      2.05391e+06       27          3.48503          3.42718     16.74m
   2    19.99      2.67266e+10       13           2.4308          2.58527     11.96m
   3    12.47      9.96721e+06       13          1.55962          1.58239      7.97m
   4    18.36       3.2885e+07       13          1.55991          1.57979     12.35m
   5    16.84      9.28862e+06       18          1.53076          1.53807     10.16m
   6    18.48          18241.2       18          1.52855          1.55796     11.24m
   7    14.55          16603.9       18          1.52596           1.5813      9.77m
   8    11.01           596001       15          1.52178           1.5448  

In [ ]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.849711232645084
R^2: 0.9900013654137402


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.4065 * S**(-0.2) *Re **(0.64)

In [ ]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.5671402704718504 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 209.95313653497948 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 13.461254276672511 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 22.902589897738164*100 


### Entrenar agregando potencias V4

In [ ]:
# * S ^(-0.2) * Pr
def pot1(x1,x2):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0), x1**(-0.2)*x2, 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=2)

# 0.5*Re^(0.64)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) , 0.5*x1**(0.64), 0)
    return result

pot2_fn = make_function(function=pot2,
                        name='pot2',
                        arity=1)


In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    24.85      3.75924e+11        8          2.27472          2.31835     19.15m
   1    18.24           144130       35          1.90703          1.93398     12.97m
   2    25.53            28689       11          1.81943          1.84413     17.83m
   3    45.30          15447.7       11          1.81137          1.91659     24.54m
   4    27.54          6584.38       16          1.55501          1.52351     15.98m
   5    21.27      1.83745e+07       16          1.55029          1.56591     12.34m
   6    16.35          7812.12       16          1.54852          1.61831     10.56m
   7    12.99           3417.8       16          1.54867          1.58055      8.25m
   8    12.03          2422.51       16          1.54599          1.60465  

In [ ]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.920381752448901
R^2: 0.989855664222547


In [ ]:
X_test

,K,Rem,prandtl
0,0.6,2824.604561,0.709843
1,0.6,2824.604561,0.709843
2,0.6,2352.302465,0.709842
3,0.6,2000.632597,0.709842
4,0.6,1878.735604,0.709842
...,...,...,...
8605,1.2,9267.885175,0.707768
8606,1.2,8959.387980,0.707705
8607,1.2,8691.558442,0.707625
8608,1.2,8537.950633,0.707554


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.425 * S**(-0.2) *(Re*Pr) **(0.64)

,nusselt
0,61.165728
1,52.824696
2,50.527730
3,48.422178
4,48.421871
...,...
8605,108.036259
8606,104.910067
8607,105.201331
8608,107.374645


In [ ]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9834064526321625 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 8.048490257841516 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 2.204764220777838 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 3.8382421591454543*100 


### Entrenar agregando potencias V4

In [ ]:
# 0.5 * S ^(-0.2) * Pr
def pot1(x1,x2):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0),0.5*x1**(-0.2)*x2, 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=2)

# Re^(0.64)
def pot2(x1):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) , x1**(0.64), 0)
    return result

pot2_fn = make_function(function=pot2,
                        name='pot2',
                        arity=1)

In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=123,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn,pot2_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

d:\env\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    24.85      2.63625e+11       28          2.13181          2.13015     17.50m
   1    22.52       1.0166e+07       24          1.71166          1.70853     14.45m
   2    28.67           123170       24          1.70518          1.76685     19.13m
   3    27.14          10805.9       11            1.544           1.5088     15.15m
   4    26.88          8492.31       11          1.53039          1.63125     14.78m
   5    22.10      1.83723e+07       11          1.53442          1.59497     11.13m
   6    14.49      1.95097e+06       11          1.53082          1.62744      7.79m
   7    10.37          6396.48       11          1.52825          1.65051      6.65m
   8    10.03          2480.13       11          1.52442            1.685  

In [ ]:
# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.744489424390825
R^2: 0.9902183008890233


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.32086 * S**(-0.2) * Re**(0.64) * Pr**(-0.0806)

In [ ]:
# Calcular cdrag_pred y cdrag_gp_pred en vectores separados
cdrag_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
cdrag_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para cdrag
r2_cdrag = r2_score(y_test['nusselt'], cdrag_pred)
mse_cdrag = mean_squared_error(y_test['nusselt'], cdrag_pred)

# Calcular MAE y MAPE para cdrag
mae_cdrag = mean_absolute_error(y_test['nusselt'], cdrag_pred)
mape_cdrag = mean_absolute_percentage_error(y_test['nusselt'], cdrag_pred)

# Calcular R^2 y MSE para cdrag_gp
r2_cdrag_gp = r2_score(y_test['nusselt'], cdrag_gp_pred)
mse_cdrag_gp = mean_squared_error(y_test['nusselt'], cdrag_gp_pred)

# Calcular MAE y MAPE para cdrag_gp
mae_cdrag_gp = mean_absolute_error(y_test['nusselt'], cdrag_gp_pred)
mape_cdrag_gp = mean_absolute_percentage_error(y_test['nusselt'], cdrag_gp_pred)

#  Mostrar Metricas

print(f"R^2 cdrag original: {r2_cdrag}, R^2 cdrag GPLearn: {r2_cdrag_gp} ")
print(f"MSE cdrag original: {mse_cdrag}, MSE cdrag GPLearn: {mse_cdrag_gp} ")
print(f"MAE cdrag original: {mae_cdrag}, MAE cdrag GPLearn: {mae_cdrag_gp} ")
print(f"MAPE cdrag original: {mape_cdrag*100}*100, MAPE cdrag GPLearn: {mape_cdrag_gp*100}*100 ")

R^2 cdrag original: 0.9509992536047758, R^2 cdrag GPLearn: 0.9901893887939061 
MSE cdrag original: 23.767192225174124, MSE cdrag GPLearn: 4.75851287041635 
MAE cdrag original: 4.174721418276441, MAE cdrag GPLearn: 1.5608072500947368 
MAPE cdrag original: 7.197154998836116*100, MAPE cdrag GPLearn: 2.719390533868199*100 


### Experimento: ajuste automatico de constante

In [ ]:
# X4 * S ^(-0.2) * Re ^0.64 * Pr
def pot1(x1, x2, x3, A):
    con = (A>0) & (A<1)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0) & (x3>0) & con, A*x1**(-0.2)* x2 **(0.64) * x3, 0)
    return result

pot1 = make_function(function=pot1,
                        name='pot1',
                        arity=4)

In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20,
                        stopping_criteria=0.01,
                        p_crossover=0.7,
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05,
                        p_point_mutation=0.1,
                        max_samples=0.9,
                        verbose=1,
                        parsimony_coefficient=0.01,
                        random_state=0,
                        function_set=[pot1])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   759.06          424.947        9          8.75726           8.7988    594.60m
   1    11.69          4408.86        9          7.71734          7.90505     11.42m
   2     7.97            27399        9          2.51058          2.61845      8.41m
   3     8.36          2911.96        5          1.92589          1.87022      8.27m
   4     9.80           148170        9          1.58864          1.57872      9.17m
   5     9.82          41887.8        9          1.58689          1.59444      8.20m
   6    10.09          33362.6        9          1.57781           1.6762      7.84m
   7     9.00          6850.15        9          1.56749          1.63974      6.74m
   8     9.26          12757.4        9          1.55632          1.56564  

In [ ]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.694393322970937
R^2: 0.9903215838657338


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.30912*(S*Pr)**(-0.2)*Re**(0.64)

In [ ]:
# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

#  Mostrar Métricas

print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100}%, MAPE nusselt GPLearn: {mape_nusselt_gp*100}% ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.990348112149519 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 4.681526114478963 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.5603540746549476 
MAPE nusselt original: 7.197154998836116%, MAPE nusselt GPLearn: 2.7361180199534223% 


### Experimento: Ajuste automatico de exponente

In [ ]:
# x1* A
def pot1(x1, A):
    con = (x1>0)
    con2 = (A<1) & (A>-1)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con & con2, np.power(x1,A), 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=2)

In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20,
                        stopping_criteria=0.01,
                        p_crossover=0.7,
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05,
                        p_point_mutation=0.1,
                        max_samples=0.9,
                        verbose=1,
                        parsimony_coefficient=0.01,
                        random_state=42,
                        function_set=['mul',pot1_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


<ipython-input-54-80b9b0bf0bf0>:6: RuntimeWarning: overflow encountered in power
  result = np.where(con & con2, np.power(x1,A), 0)


   0    29.86       9.0327e+12        5          5.09411          5.08621     14.03m
   1    10.02      3.95072e+06        5          5.05824          5.40902      6.17m
   2    10.87      2.59763e+07        7          4.10632          3.97934      5.33m
   3     8.87      1.76421e+06        7           4.0847          4.17397      4.68m
   4     5.84           539341        7          4.06301          4.36916      3.99m
   5     5.57      9.43835e+06        7          4.06659          4.33688      3.58m
   6     6.21      1.48285e+06        7          4.06663          4.33656      3.18m
   7     6.88      1.79694e+06        7          3.93202           3.9537      2.62m
   8     7.02      1.37634e+10        7          3.92105          4.05244      2.32m
   9     7.10          86738.1        7          3.91721          4.08703      2.19m
  10     7.12          18056.8        7          3.90792           4.1706      2.01m
  11     7.04           180093        7          3.90118         

In [ ]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 31.9022198519111
R^2: 0.9342272924122517


### Experimento: Ajuste Automatico de constante V2

In [ ]:
# x1** A *x2
def pot1(x1,x2, A):
    con = (x1>0)
    con2 = (A<1) & (A>-1)
    con3 = (x2>0)
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where(con & con2, x2*np.power(x1,A), 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=3)

In [ ]:
# Modelo
model = SymbolicRegressor(population_size=5000,
                        generations=20,
                        stopping_criteria=0.01,
                        p_crossover=0.7,
                        p_subtree_mutation=0.1,
                        p_hoist_mutation=0.05,
                        p_point_mutation=0.1,
                        max_samples=0.9,
                        verbose=1,
                        parsimony_coefficient=0.01,
                        random_state=0,
                        function_set=['mul',pot1_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left


<ipython-input-47-1b1940ad24f9>:7: RuntimeWarning: overflow encountered in power
  result = np.where(con & con2, x2*np.power(x1,A), 0)
<ipython-input-47-1b1940ad24f9>:7: RuntimeWarning: overflow encountered in multiply
  result = np.where(con & con2, x2*np.power(x1,A), 0)


   0    78.74      1.56333e+27       35          11.3756          11.3252     28.76m
   1    11.24      8.79388e+08       25          4.01821          4.19878      5.56m
   2    12.61      3.95202e+12       15          2.97036          2.99578      5.60m
   3    18.50      3.00022e+09       15          2.96261          3.06557      8.58m
   4    21.36      9.23829e+07       15          2.95465          3.13724      7.44m
   5    19.79      1.60217e+06       18          1.99733          2.05634      5.74m
   6    16.60       1.0107e+07       34          1.84447          1.88272      5.02m
   7    17.21      1.29466e+11       24          1.81553          1.87377      4.42m
   8    19.10      4.74083e+09       24          1.81152          1.90984      4.48m
   9    24.42       3.4281e+08       42          1.79785          1.86016      4.64m
  10    22.14      4.06209e+06       24           1.8016          1.86552      4.13m
  11    18.66       1.0723e+07       22          1.80012         

In [ ]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 6.7616908401481925
R^2: 0.9860594429951187


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.484 * Pr * 0.287**(0.311*S*Pr) * Re**0.670

In [ ]:
# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

#  Mostrar Métricas

print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100}%, MAPE nusselt GPLearn: {mape_nusselt_gp*100}% ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9862733804514594 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 6.657923183059813 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.7929337892268358 
MAPE nusselt original: 7.197154998836116%, MAPE nusselt GPLearn: 3.1159537690684957% 


### Experimento: Ajuste automatico de constante V3

In [ ]:
def pot1(x1, x2, x3, x4):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.where((x1 > 0) & (x2>0) & (x3>0), x4*x1**(-0.2)* x2 **(0.64) * x3, 0)
    return result

pot1_fn = make_function(function=pot1,
                        name='pot1',
                        arity=4)

In [ ]:
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=42,
                           function_set=[pot1_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0   757.77      8.83101e+16        5          4.31785          4.39546    604.62m
   1    13.14      2.30813e+12        5          2.81652          2.79483     12.71m
   2    10.76      7.16365e+08       37          1.77045           1.6835     10.87m
   3    11.23      4.63401e+19       37          1.76174          1.76185     10.10m
   4     6.86      5.74908e+07       37          1.75492          1.82328      6.88m
   5     7.27      1.97571e+08       37          1.75144           1.8546      6.60m
   6    10.09      5.50028e+20       37           1.7493          1.87379      8.21m
   7    11.90      8.84481e+08       37          1.75118          1.85693      8.10m
   8    10.48      1.32567e+09        9          1.60515          1.61113  

In [ ]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.702760642569644
R^2: 0.9903043329889041


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return 0.484 * Pr * 0.287**(0.311*S*Pr) * Re**0.670

### Experimento: Ajuste automatico de constante V4

In [ ]:
model = SymbolicRegressor(population_size=5000,
                           generations=20,
                           stopping_criteria=0.01,
                           p_crossover=0.7,
                           p_subtree_mutation=0.1,
                           p_hoist_mutation=0.05,
                           p_point_mutation=0.1,
                           max_samples=0.9,
                           verbose=1,
                           parsimony_coefficient=0.01,
                           random_state=42,
                           function_set=['add', 'sub', 'mul', 'div',pot1_fn])

model.fit(X_train, y_train)

# Obtener la expresión simbólica
expression = model._program
print("Expresión:", expression)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    75.65      4.88859e+23        7          4.50603          4.42802     26.36m
   1    27.36      2.65266e+13        7          4.48752          4.59462     10.02m
   2    41.64      5.27191e+09       19          1.72347          1.66648     13.85m
   3    26.97      2.14706e+11       17          1.71212          1.76924     10.19m
   4    17.17      2.62789e+11       19          1.70961          1.79233      7.34m
   5    23.74      1.20825e+07       19          1.66083          1.69868      8.03m
   6    25.07      1.28172e+09       15          1.62325          1.66641      7.64m
   7    16.90      6.33915e+09       15          1.62054          1.69074      4.78m
   8    13.66      9.49802e+10       15          1.59009          1.60537  

In [ ]:
y_pred = model.predict(X_test)

# Calcular métricas
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R^2:", r2)

MSE: 4.787532791681305
R^2: 0.9901295585123633


In [ ]:
def nusselt(S, Re, Pr):
    return 0.5 * S **(-0.2) * Re **(0.64) * Pr

def nusselt_gp(S, Re, Pr):
    return -0.809424*S*Pr + 0.75204*S**(-0.2)* Re**(0.64)*(-0.264+Pr)

In [ ]:
# Calcular nusselt_pred y nusselt_gp_pred en vectores separados
nusselt_pred = X_test.apply(lambda row: nusselt(row['K'], row['Rem'], row['prandtl']), axis=1).values
nusselt_gp_pred = X_test.apply(lambda row: nusselt_gp(row['K'], row['Rem'], row['prandtl']), axis=1).values

# Calcular R^2 y MSE para nusselt
r2_nusselt = r2_score(y_test['nusselt'], nusselt_pred)
mse_nusselt = mean_squared_error(y_test['nusselt'], nusselt_pred)

# Calcular MAE y MAPE para nusselt
mae_nusselt = mean_absolute_error(y_test['nusselt'], nusselt_pred)
mape_nusselt = mean_absolute_percentage_error(y_test['nusselt'], nusselt_pred)

# Calcular R^2 y MSE para nusselt_gp
r2_nusselt_gp = r2_score(y_test['nusselt'], nusselt_gp_pred)
mse_nusselt_gp = mean_squared_error(y_test['nusselt'], nusselt_gp_pred)

# Calcular MAE y MAPE para nusselt_gp
mae_nusselt_gp = mean_absolute_error(y_test['nusselt'], nusselt_gp_pred)
mape_nusselt_gp = mean_absolute_percentage_error(y_test['nusselt'], nusselt_gp_pred)

#  Mostrar Métricas

print(f"R^2 nusselt original: {r2_nusselt}, R^2 nusselt GPLearn: {r2_nusselt_gp} ")
print(f"MSE nusselt original: {mse_nusselt}, MSE nusselt GPLearn: {mse_nusselt_gp} ")
print(f"MAE nusselt original: {mae_nusselt}, MAE nusselt GPLearn: {mae_nusselt_gp} ")
print(f"MAPE nusselt original: {mape_nusselt*100}%, MAPE nusselt GPLearn: {mape_nusselt_gp*100}% ")

R^2 nusselt original: 0.9509992536047758, R^2 nusselt GPLearn: 0.9901320723045455 
MSE nusselt original: 23.767192225174124, MSE nusselt GPLearn: 4.7863135085804815 
MAE nusselt original: 4.174721418276441, MAE nusselt GPLearn: 1.542455482654369 
MAPE nusselt original: 7.197154998836116%, MAPE nusselt GPLearn: 2.6848392142824777% 


### Experimento: Ajuste automatico de constante V5